# Generate onnx models at multiple input scales

In [1]:
import copy
import random
import os
import sys

import albumentations as A
import albumentations.augmentations.functional as F
from albumentations.pytorch import ToTensorV2
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import JaccardIndex
from statistics import mean
import torchvision.transforms as T
import torch.onnx
import onnx
import onnxruntime
from torchsummary import summary


from utils.lr_scheduler import PolynomialLRDecay
from models.experiments.fast_scnn_mod import FastSCNN as fastscnn_mod

from utils.dataset import PackagesDataset, PackagesInferenceDataset
from utils.evaluation import Evaluate
cudnn.benchmark = True

In [2]:
def remove_initializer_from_input(model_path):

    onnxmodel = onnx.load(model_path+".onnx")
    if onnxmodel.ir_version < 4:
        print("Model with ir_version below 4 requires to include initilizer in graph input")
        return

    inputs = onnxmodel.graph.input
    name_to_input = {}
    for input in inputs:
        name_to_input[input.name] = input

    for initializer in onnxmodel.graph.initializer:
        if initializer.name in name_to_input:
            inputs.remove(name_to_input[initializer.name])
    out_path = model_path+"_noinit.onnx"
    onnx.save(onnxmodel, out_path)

In [3]:
dimension_multipliers =[1.0, 0.5, 0.4375, 0.375, 0.3125, 0.25, 0.1875]

In [4]:
c_dir = os.getcwd()
model_path = os.path.join(c_dir, "model_checkpoints/experiments/fastscnn_a1.0t6r3pp0.pt")
model = fastscnn_mod(in_channels=3, num_classes=1, a=1.0, t=6, r=3, pp=False).cuda()
model.load_state_dict((torch.load(model_path)))
model.eval()
print()


The model has been initialized with parameters:
	 Width multiplier a =  1.0
	 Bottleneck expansion rate t =  6
	 Bottleneck block repetition r =  3
	 Presence of Pyramid Pooling module pp =  False



In [5]:
output_path = os.path.join(c_dir, "model_checkpoints/experiments/no_pretraining")

In [7]:
for m in dimension_multipliers:
    #put the model in inference mode
    onnx_path = output_path+"/fastscnn_"+str(int(m*512))+"_noPT"
    #generate dummy input for onnx export
    x = torch.randn(1, 3, int(m*512), int(m*512), requires_grad=True).cuda()
    torch_out = model(x)

    # Export the model
    torch.onnx.export(model,                         # model being run
                      x,                             # model input (or a tuple for multiple inputs)
                      onnx_path+".onnx",            # where to save the model (can be a file or file-like object)
                      export_params=True,            # store the trained parameter weights inside the model file
                      opset_version=11,              # the ONNX version to export the model to
                      do_constant_folding=True,      # whether to execute constant folding for optimization
                      input_names = ['input'],       # the model's input names
                      output_names = ['output'],     # the model's output names
                      operator_export_type=torch.onnx.OperatorExportTypes.ONNX,
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})
    remove_initializer_from_input(onnx_path)
    print("Onnx model saved")
    print()

Onnx model saved

Onnx model saved

Onnx model saved

Onnx model saved

Onnx model saved

Onnx model saved

Onnx model saved

